In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import time
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
print(tf.version) # tensorflow 2.0
pd.options.display.max_rows = 8 # will use 8 by default for count, mean, std ... max
pd.options.display.max_columns = 9
pd.options.display.float_format = '{:.6f}'.format
pd.set_option('mode.chained_assignment', None)

<module 'tensorflow_core._api.v2.version' from 'c:\\users\\r-k-l\\appdata\\local\\programs\\python\\python37\\lib\\site-packages\\tensorflow_core\\_api\\v2\\version\\__init__.py'>


In [2]:
df = pd.read_csv("Dataset_Github_Labeled.csv")

# for Dataset_Github_Labeled.csv only: change y in the csv file to be assigned to one of three classes: High-grade, Low-grade, Normal
for i in range (0,324): # 0 - 323, same size as x
    #print(type(y[i]))
    if df['class'][i].startswith('High-grade'):  # if the last column contains text "High-grade", etc below.
        df['class'][i] = 'High-grade'
    elif df['class'][i].startswith('Low-grade'):
        df['class'][i] = 'Low-grade'
    elif df['class'][i].startswith('Normal'):
        df['class'][i] = 'Normal'

# https://stackoverflow.com/questions/38250710/how-to-split-data-into-3-sets-train-validation-and-test
# x represents attributes, y represents class label
training, validation, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))]) # 60% test, 20% validation, 20% test split.
x_train = training.drop(['class'], axis=1)
y_train = training['class']
x_validation=validation.drop(['class'], axis=1)
y_validation=validation['class']
x_test=test.drop(['class'], axis=1)
y_test=test['class']
# Encode class label y to be 0, 1, or 2
from sklearn.preprocessing import LabelEncoder
lbl_encoder = LabelEncoder()
y_train= lbl_encoder.fit_transform(y_train)
y_test= lbl_encoder.fit_transform(y_test)
y_validation= lbl_encoder.fit_transform(y_validation)
del training, validation, test # clear memory of variables not needed

In [3]:
# def construct_feature_columns(input_features_DataFrame):
#   """Construct the TensorFlow Feature Columns.
#   Args:
#     input_features: DataFrame with the names of the numerical input features to use.
#   Returns:
#     A set of tf numeric feature columns
#   """
#   tensorSet = ([])
#   for elem in input_features_DataFrame:
#     tensorSet.append(tf.feature_column.numeric_column(key=str(elem), dtype=tf.dtypes.float64) ) # where elem is a str feature label
#   return tensorSet

# # Create the input function for training + evaluation. boolean = True for training.
# def input_fn(features, labels, training=True, batch_size=32 ):
#     dataf = tf.data.Dataset.from_tensor_slices((dict(features), labels))
#     if training:
#         dataf = dataf.shuffle(100).repeat() # shuffle ~<half the data
#     return dataf.batch(batch_size=batch_size)

In [4]:
# # Construct feature columns, which is a list of tensors with the feature names, shown below. 
# feature_columns=construct_feature_columns(df.drop(['class'], axis=1).head(0))
# len(feature_columns)

In [5]:
# model = tf.estimator.BoostedTreesClassifier(feature_columns, n_batches_per_layer = 1,  n_classes=3, pruning_mode=None)
# model.train(input_fn=lambda: input_fn(features=x_train, labels=y_train, training=True), max_steps=1000)
# validation_results = model.evaluate(input_fn=lambda: input_fn(features=x_validation, labels=y_validation, training=False), steps=1)
# testing_results = model.evaluate(input_fn=lambda: input_fn(features=x_test, labels=y_test, training=False), steps=1)
# print("Validation results: ")
# print(validation_results)
# print("Testing results: ")
# print(testing_results)

In [6]:
# train 
svclassifier = SVC(kernel='linear', probability=True)
svclassifier.fit(x_train, y_train)

# predict validation
svm_predictions = svclassifier.predict(x_validation)


# evaluate validation
v_acc = svclassifier.score(x_validation, y_validation) # returns accuracy

# predict test
svm_predictions = svclassifier.predict(x_test)

# evaluate test
t_acc = svclassifier.score(x_test, y_test) # returns accuracy

print("Validation accuracy: ", v_acc)
print("Test accuracy: ", t_acc)
print("Average: ", round((v_acc+t_acc)/2, 2))


Validation accuracy:  0.8615384615384616
Test accuracy:  0.8769230769230769
Average:  0.87


In [7]:
# print(confusion_matrix(y_test,svm_predictions))
# print(classification_report(y_test,svm_predictions, target_names=["High-grade", "Medium-grade", "Normal"]))